In [1]:
import pandas as pd
import requests

# 1. Steam API에서 전체 앱 목록 받아오기
def get_all_steam_apps():
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    app_list = response.json()['applist']['apps']
    df_apps = pd.DataFrame(app_list)
    df_apps.to_csv("steam_applist.csv", index=False, encoding="utf-8-sig")  # 로컬 저장
    return df_apps

# 2. 기존 raw 데이터 GitHub에서 불러오기
def load_raw_data():
    urls = {
        "block_1": "https://raw.githubusercontent.com/Kim-TaeWook/steam-analysis/refs/heads/main/data_raw/steam_games_block_1.csv",
        "block_2": "https://raw.githubusercontent.com/Kim-TaeWook/steam-analysis/refs/heads/main/data_raw/steam_games_block_2.csv",
        "partial": "https://raw.githubusercontent.com/Kim-TaeWook/steam-analysis/refs/heads/main/data_raw/steam_game_data_partial.csv"
    }
    df1 = pd.read_csv(urls["block_1"])
    df2 = pd.read_csv(urls["block_2"])
    df3 = pd.read_csv(urls["partial"])
    return df1, df2, df3

# 3. 이름 수정 함수
def fix_names(df_raw, df_apps):
    name_map = dict(zip(df_apps["appid"], df_apps["name"]))
    df_raw["name_fixed"] = df_raw["appid"].map(name_map)
    df_raw["name"] = df_raw["name_fixed"].combine_first(df_raw["name"])
    return df_raw.drop(columns=["name_fixed"])

# 4. 전체 실행
df_apps = get_all_steam_apps()
df1, df2, df3 = load_raw_data()

df1_fixed = fix_names(df1, df_apps)
df2_fixed = fix_names(df2, df_apps)
df3_fixed = fix_names(df3, df_apps)

# 5. 저장 (개별 + 병합본)
df1_fixed.to_csv("steam_games_block_1_fixed.csv", index=False, encoding="utf-8-sig")
df2_fixed.to_csv("steam_games_block_2_fixed.csv", index=False, encoding="utf-8-sig")
df3_fixed.to_csv("steam_game_data_partial_fixed.csv", index=False, encoding="utf-8-sig")

df_all = pd.concat([df1_fixed, df2_fixed, df3_fixed], ignore_index=True)
df_all.to_csv("steam_data_fixed_combined.csv", index=False, encoding="utf-8-sig")

print("✅ 이름 수정 완료 및 저장 완료")


✅ 이름 수정 완료 및 저장 완료
